In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from IPython.display import display, HTML

In [3]:
import locale
locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

In [4]:
df = pd.read_csv('~/Documents/industry.csv')

In [5]:
# Clean up column names
df.rename(columns = lambda x: x.strip(), inplace=True)

In [6]:
def convert_accounting_number(x, dash_values = np.nan, negative_values = None):
    x = x.strip()
    x = x.replace(',', '')
    if x == '-':
        return dash_values
    factor = 1
    if x[0] == '(':
        x = x[1:-1]
        if negative_values is None:
            return -int(x)
        else:
            return negative_values(int(x))
    else:
        return int(x)

df["Total Cost String"] = df["Total Cost"]
df["Total Cost"] = df["Total Cost String"].apply(convert_accounting_number)
df["Total Cost Subtract Negative"] = df["Total Cost String"].apply(convert_accounting_number, negative_values = lambda x: -x)
df["Total Cost Add Negative"] = df["Total Cost String"].apply(convert_accounting_number, negative_values = lambda x: x)
df["Total Cost Ignore Negative"] = df["Total Cost String"].apply(convert_accounting_number, negative_values = lambda x: 0.0)

In [7]:
df["Sponsor Name"] = df["Sponsor Name"].apply(str.upper)

In [8]:
df.columns

Index([u'FY', u'OP Date', u'Sponsor Name', u'Spon Code', u'Sponsor Award#',
       u'Dept Code', u'Dept Name', u'VC Name', u'Total Cost', u'UCSD Award#',
       u'Seq #', u'Base UCSD#', u'PI', u'Title', u'Award Type',
       u'Activity Type', u'Designation', u'Start', u'End',
       u'Total Cost String', u'Total Cost Subtract Negative',
       u'Total Cost Add Negative', u'Total Cost Ignore Negative'],
      dtype='object')

In [9]:
df["Total Cost"].sum()

632716403.0

In [10]:
df["Total Cost Ignore Negative"].sum()

652268128.0

In [11]:
sponsors = df["Sponsor Name"].sort_values().unique()
print(len(sponsors))
for s in sponsors:
    print s

938
AASTROM BIOSCIENCES, INC.
AB SCIENCE SA (FRANCE)
ABBOTT DIABETES CARE, INC.
ABBOTT LABORATORIES
ABBOTT LABORATORIES (INCL PERCLOSE)
ABBVIE INC.
ABREOS BIOSCIENCES
ABT ASSOCIATES INC.
AC IMMUNE (SWITZERLAND)
ACCERA, INC.
ACCUMETRICS
ACELRX PHARMACEUTICALS, INC.
ACERTA PHARMA LLC
ACORDA THERAPEUTICS
ACTELION
ACTELION 
ACTELION (INCL HESPERION LTD.) (SWI
ADHERON THERAPEUTICS
ADNEXUS
ADURO BIOTECH
ADVANCED BIOHEALING
ADVANCED BRAIN MONITORING, INC.
ADVANCED CLINICAL
ADVANCED MATERIALS AND DEVICES, INC
ADVANCED NEUROMODULATION SYSTEMS, I
ADYB ENGINEERED FOR LIFE, INC
AERIE PHARMACEUTICALS, INC.
AERODYNE RESEARCH, INC.
AERPIO THERAPEUTICS
AGA MEDICAL CORPORATION
AGILENT TECHNOLOGIES
AGILTRON INC.
AIRES PHARMACEUTICALS, INC.
AJINOMOTO CO., INC.
AKEBIA THERAPEUTICS, INC.
AKONNI BIOSYSTEMS
ALCON LABORATORIES
ALCON, INC.
ALERE MEDICAL INCORPORATED
ALERE MEDICAL INCORPORATED (INCLUDE
ALEXION PHARMACEUTICALS, INC.
ALLERGAN INC.
ALLERGAN INC. (PHARMACEUTICALS & OP
ALLERGEN RESEARCH CORPORATION


In [12]:
def rows_by_sponsor(data, sponsor, sponsor_name_column = 'Sponsor Name'):
    mask = [sponsor in x for x in df[sponsor_name_column]]
    return data[mask]

In [13]:
sponsors_to_analyze = [
    "LOCKHEED MARTIN",
    "NORTHROP GRUMMAN",
    "GENERAL ATOMICS",
    "GENERAL DYNAMICS",
    "GENERAL ELECTRIC",
    "LOS ALAMOS NATIONAL SECURITY",
    "HEWLETT-PACKARD",
    "ROCKWELL COLLINS",
    "CACI INTL",
    "SAIC",
    "RAYTHEON",
    "BOEING",
    "BOOZ-ALLEN",
]

new_data = []

for name in sponsors_to_analyze:
    rows = rows_by_sponsor(df, name)
    cost = rows["Total Cost Ignore Negative"].sum()
    if pd.isnull(cost):
        cost = 0
    cost_str = "$" + locale.format("%d", cost, grouping=True)
    x, y = zip(*[
        ("Sponsor Name", name),
        ("Number of Entries", len(rows)),
        ("Total Cost", cost_str),
        ("Total Cost Numeric", cost)
    ])
    new_data.append(pd.Series(y, index=x))

summary = pd.DataFrame.from_records(new_data)
summary = summary.sort_values('Total Cost Numeric', ascending=False)
summary.drop('Total Cost Numeric', axis=1, inplace=True)

In [14]:
display(HTML(summary.to_html(index=False)))

Sponsor Name,Number of Entries,Total Cost
LOS ALAMOS NATIONAL SECURITY,62,"$10,192,586"
GENERAL ATOMICS,82,"$4,981,984"
GENERAL ELECTRIC,33,"$2,918,317"
LOCKHEED MARTIN,28,"$2,072,665"
SAIC,38,"$1,654,935"
RAYTHEON,22,"$884,693"
ROCKWELL COLLINS,13,"$779,113"
BOEING,3,"$320,000"
HEWLETT-PACKARD,5,"$240,602"
NORTHROP GRUMMAN,7,"$183,364"


In [15]:
sponsor_name = "CACI INTL"
subset = rows_by_sponsor(df, sponsor_name)
for row_index, row in subset.iterrows():
    print(row["Title"])
subset

Chronopolis Digital Preservation Archive and Demonstration Project: Extension


,FY,OP Date,Sponsor Name,Spon Code,Sponsor Award#,Dept Code,Dept Name,VC Name,Total Cost,UCSD Award#,...,Title,Award Type,Activity Type,Designation,Start,End,Total Cost String,Total Cost Subtract Negative,Total Cost Add Negative,Total Cost Ignore Negative
435,11,02/02/11,CACI INTL INC. (INCL CACI DYNAMICS,5558,Sub SA-05-0045-D02 (GS09K99BHD0006),170,SDSC San Diego Supercomputer Center,ACADEMIC AFF,NaN,081475-001,...,Chronopolis Digital Preservation Archive and D...,Administrative Amendment,Basic Research,Sub-award Contract,12/01/07,08/31/10,-,NaN,NaN,NaN
